# 编写注意力机制
- ch2中没有这个，只是“预测”已有文本的下一位是什么，所以需要本节的attention mechanisms.


In [3]:
from importlib.metadata import version   #检测本节所需要的库

print("torch version:", version("torch"))

torch version: 2.4.1


- 从简化的注意力机制 >> 正常的注意力机制 >> 带有因果关系的注意力机制 >> 同时思考

## 1.用当前模型处理一些长序列存在的问题
- 不同语言的语法结构不同，如果按照顺序翻译，容易出现问题
- 所以需要引入一个基于神经网络的中间层(hidden state)，对数据顺序进行处理

## 2.利用注意力机制捕获数据依赖

- 通过注意力机制，文本生成解码器能够有选择的处理数据（基于一些数据的重要性比其他的要高）
- self-attention 就是使序列中的每一位都参与并且和同类序列比较来决定相关性，从而加强每一个数据的特征。

## 3.通过自注意力来注意输入的不同部分

### 3.1 一个不使用训练权重的自注意力机制

- 为了便于理解，但真正运用的自注意力机制还是需要考虑权重的

- 将每一个已经转化为嵌入的token，映射到一个另外一个向量。这个向量和token是同维度的，但是它会计算从1号到全部的token的权重（与当前对应的token之间的关系权重），将值存在里面，构成一个上下文关系型向量（具体内容是取决于当前给定的文本的）

- 第一步，计算注意力分数$ \omega_{ij} = x^{(j)} · q^{(i)\top} $ （注意力分数是未归一化的结果，如果归一化了就是注意力权重）
- 每一个x会被映射到一个q，这里假定$q^{(i)} == x^{(i)}$ .然后需要利用这里的$q^{(i)}$,计算出所有其他token与当前i号token的点积（获得注意力分数）

In [13]:
import torch

inputs = torch.tensor(                               #假定此时已经获得了一个token embedding如左边所示
  [[0.43, 0.15, 0.89], # Your     (x^1)              #本书按照惯例，每一行代表一个单词(/token)，列代表嵌入向量维数
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [14]:
query = inputs[1]  # 现在我们假设计算z2的权重，用x2作为查询向量q

attn_scores_2 = torch.empty(inputs.shape[0]) #初始化，获得一个token词元规格的空值
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # 向量点积，因为这里例子都是一元向量，所以不用考虑转置之类的

print(attn_scores_2)   #打印所有注意力分数之和。

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [15]:
res = 0.                                    #记录注意力分数的第一位，用于验证上方的计算结果

for idx, element in enumerate(inputs[0]):   #对于第一个token中的所有维度
    res += inputs[0][idx] * query[idx]      #res表示每一个维度和query向量中对应维度下的值相乘

print(res)                                  #实际上就是实现两个向量的点积，只是拆开来写了。
print(torch.dot(inputs[0], query))          #将x1向量和query向量直接点积的结果

tensor(0.9544)
tensor(0.9544)


- 第二步，将上述获得的注意力分数进行归一化 （而实际操作中，更多使用一种softmax function的方法进行归一）

In [17]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()    #将单个数值除以总数，进行归一化

print("Attention weights:", attn_weights_2_tmp)  #打印当前所有数值
print("Sum:", attn_weights_2_tmp.sum())   #打印总和

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [18]:
#softmax function 的简易实现
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)   #归一化函数，传入待归一的注意力分数张量，返回归一化的结果

attn_weights_2_naive = softmax_naive(attn_scores_2) #调用函数

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [19]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)   #上述自定义的softmax_naive函数在处理大规模或者特别小的数据的时候，可能会发生溢出，所以最好用pytorch自带的softmax函数

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 第三步，通过将每个$x^{(i)}$与当前的注意力注意力参数$a^{(ji)}$相乘，求和，得到上下文向量$ z^{(j)}$

In [21]:
query = inputs[1] # 假定我们当前求的是第二个参数的注意力向量。

context_vec_2 = torch.zeros(query.shape)  #获得与这个向量同规模的初始化向量。
for i,x_i in enumerate(inputs):  
    context_vec_2 += attn_weights_2[i]*x_i   #将i号注意力参数与i号输入token嵌入相乘，求和

print(context_vec_2) #得到最终的结果 --> 注意力向量 z^2

tensor([0.4419, 0.6515, 0.5683])


### 3.2计算所有输入token的注意力权重

- 计算一个上下文向量：1.先计算出注意力分数 2.归一化为注意力权重 3.通过点积求和来计算出最终的上下文向量
- 3.1中计算了单个向量，现在要计算出所有token对应的上下文向量


In [24]:
attn_scores = torch.empty(6, 6)                  #创建一个存储注意力分数的矩阵。它的规模为(张量总数x张量总数)，因为它反应的是一个张量和其他所有张量之间的关系

for i, x_i in enumerate(inputs):                 #对每一行的这一个向量
    for j, x_j in enumerate(inputs):             #先计算其他所有向量与该向量的点积。
        attn_scores[i, j] = torch.dot(x_i, x_j)  #逐步获得了所有向量的注意力分数

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [25]:
attn_scores = inputs @ inputs.T             #使用矩阵乘法，获得和上述同等的效果
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [26]:
attn_weights = torch.softmax(attn_scores, dim=-1)  #将上述获得的注意力分数进行归一化，dim=-1表示的是在attn_scores的最后一个维度上进行归一化，而不影响其他维度。
print(attn_weights) #比如目前的矩阵是ij二维的，dim = -1 表示只在j维度上进行归一化，即只会使得每一行总和为1，而不会导致每一列总和变成1的情况

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [27]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])  #快速验证一下，每一行的注意力权重总和是否为1
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [28]:
all_context_vecs = attn_weights @ inputs  #将注意力参数与输入token嵌入相乘，获得了最后的上下文向量
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [29]:
print("Previous 2nd context vector:", context_vec_2) #验证，第二行最后得出的向量和3.2求得的应该是一样的

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 4.利用可训练的权重实现自我注意

### 4.1一步一步计算注意力权重

- 相比于上面3中的简单示例，为了可以进行权重训练，需要3个权重矩阵$ W_q , W_k , W_v $
- 这三个矩阵会将输入的token $ x^{(i)} $ 转化为 query , key ,value 三种向量（通过矩阵乘法）
- 这些query,key,value向量的维度可以和x相同，也可以不同

In [33]:
x_2 = inputs[1] #以第二个元素作为示例
d_in = 3 #输入维度（inputs中token的维度，这里为3）
d_out = 2 #输出维度

In [34]:
# requires_grad = False 是在学习的时候使用，便于数据的阅读。实际训练模型的时候，还是需要requires_grad = True

torch.manual_seed(123) #定义一个种子，使得后面的伪随机数相同，便于检查一下数据

W_query = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad = False)  #定义3个初始化为随机数的矩阵。行向量数为d_in,列向量数为d_out
W_key = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad = False)
W_value = torch.nn.Parameter(torch.rand(d_in,d_out),requires_grad = False)

In [35]:
query_2 = x_2 @ W_query  #利用矩阵乘法，将输入向量中第二位，转化成对应的查询/键/值向量
key_2 = x_2 @ W_key
value = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [36]:
keys = inputs @ W_key   #获得整个键，值矩阵。
values = inputs @ W_value

print("keys.shape : " ,keys.shape) #此时inputs矩阵的维度已经发生了变化。原本是(6,3) -->(6,2)
print("values.shape : ",values.shape)

keys.shape :  torch.Size([6, 2])
values.shape :  torch.Size([6, 2])


- 通过上述代码，获得了$ W_q , W_k , W_v $矩阵 。
-  假设当前是对2号token进行处理，那么下一步，需要将query_2向量与其他token的key向量相乘。
-  从每一个token映射到query,key,value . 然后求每一个$ x^{(i)}$ 向量对应的注意力分数的时候，就是将$ q^{(i)}$与其他所有的token的key向量进行点积

In [38]:
attn_scores_2 = query_2 @ keys.T #通过矩阵相乘，实现当前向量和所有的key向量相乘
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


- 下一步，就是将当前的注意力分数转化为注意力权重
- 这里注意力权重的计算和上方简化版本有不同，这里要将所有的注意力分数除以$\sqrt{d_k}$ .这是为了避免点积的值过大，导致的结果不稳定

In [40]:
d_k = keys.shape[1] #记录key向量的维度
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5,dim = -1)  #利用pytorch的softmax函数，将获得的注意力分数转化为注意力权重

print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


- 最后一步，就是将刚刚算出的，每一个token的注意力权重与values值相乘，得到了当前向量的上下文向量

In [42]:
context_vec_2 = attn_weights_2 @ values #2号token的上下文向量，就是刚刚通过它算出的注意力权重值与values矩阵相乘，就获得了它自身的上下文向量
print(context_vec_2)   #实际上实现的就是将每一个token对当前token的注意力权重值，与他们自身的value值相乘

tensor([0.3061, 0.8210])


### 4.2将SelfAttention包装成类

In [44]:
import torch.nn as nn 

class SelfAttention_v1(nn.Module): #以nn.Module为父类，这个是pytorch中定义神经网络模块的标准方式，实现一个自注意力层

    def __init__(self,d_in,d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in,d_out))  #初始化类，获得W_query,W_key,W_value的空矩阵，形状为（输入向量维度，输出向量维度）
        self.W_key   = nn.Parameter(torch.rand(d_in,d_out))
        self.W_value = nn.Parameter(torch.rand(d_in,d_out))

    def forward(self,x):  #覆写父类中的向前传播算法。
        keys = x @ self.W_key         #将 x（输入矩阵，形状为[token总数，单个token向量的维度]） 与 刚刚定义的空矩阵相乘，获得了keys，queries，values矩阵。
        queries = x @ self.W_query    #这里的key等为临时变量，仅适用于当次的向前传播的计算。而W_key等变量则是持久的，是模型的可训练参数，会不断更新（只是这个函数中没有提及）。
        values = x @ self.W_value

        attn_scores = queries @ keys.T #将每一个向量的query矩阵与其他向量的key矩阵相乘，即获得了注意力分数
        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5 , dim = -1) #将每一个x的注意力分数归一化，注意要除以keys向量的最后一个维度的根值。（即根号d_out）

        context_vec = attn_weights @ values #权重与values值相乘，获得最后的上下文矩阵。
        return context_vec

torch.manual_seed(123) #定义随机数种子，确保可重复性
sa_v1 = SelfAttention_v1(d_in,d_out) #将刚刚的自我注意类实例化
print(sa_v1(inputs)) #由于nn.Module中 __call__方法被重写过，可以以类似于函数调用的方法来使用这个实例。forward函数已经被写到了__call__方法内。

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


- 可以用pytorch的nn.Linear层来代替手动定义的W矩阵。当禁止偏置项的时候，它等效于矩阵乘法。
- 并且它的权重初始化要比rand()更加稳定。rand可能会出现梯度消失/爆炸的情况，导致模型难以训练。

In [46]:
class SelfAttention_v2(nn.Module): #大部分和上面写的类是一样的

    def __init__(self, d_in, d_out, qkv_bias=False): #默认偏置项为禁止的
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias) #使用Linear来生成初始化矩阵，而不是Parameter(torch.rand(xxx))
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias) #这个时候禁止偏置项，只要求其实现矩阵乘法。
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)        #这里也是Linear 的作用，它内部重写了__call__方法。 实现的功能是keys = x @ W_key + b (这里b是偏置项，本代码中禁止了）
        queries = self.W_query(x)
        values = self.W_value(x)
          
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- 这里v2 和v1 产生的结果不同，因为他们初始化的权重是不同的。


## 5.利用因果注意力来隐藏未来的词

- 在因果注意力中，当前词后面的token会被隐藏起来

### 1.使用因果掩码
- 上方实现的注意力模型，对于每一个token，它使用到了未来的，现在还没有使用的词语。通过因果掩码修饰，使得它只能依赖在它之前的词元来训练注意力
- 注意力分数 --> 注意力权重 --> 掩码注意力分数 --> 掩码注意力权重

In [51]:
queries = sa_v2.W_query(inputs)    #再利用上面的查询矩阵和键矩阵。将初始化矩阵和inputs矩阵做乘法，获得了当前的查询向量矩阵。
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5,dim = -1)  #此时都是未进行掩码操作的
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 一个最简化版本的掩码注意力权重

In [53]:
context_length = attn_scores.shape[0]   #将获取注意力分数/权重矩阵中的词元总数
mask_simple = torch.tril(torch.ones(context_length,context_length)) #获取一个只有0和1的矩阵。利用pytorch中的tril方法，创建一个对角矩阵
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [54]:
mask_simple = mask_simple*attn_weights #将刚刚获得的这个对角矩阵和已有的注意力权重矩阵相乘，获得掩码矩阵。
print(mask_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 这里归一化不便于使用softmax。因为之前已经产生注意力权重的时候，已经使用过softmax了。再一次使用它，可能会产生不必要的错误，破坏概率分布
- 最好的方法是在softmax之前使用掩码

- 为了利用已经产生的掩码分数，采用另一种方法进行归一化

In [56]:
row_sums = mask_simple.sum(dim=-1,keepdim = True) #计算每一行的和，这里dim=-1表示求和的是最后一维度（本题中就是列向量求和） keepdim表示求和后，保留列向量的维度，只是大小变成1
masked_simple_norm = mask_simple / row_sums
print(row_sums)
print(masked_simple_norm)

tensor([[0.1921],
        [0.3700],
        [0.5357],
        [0.6775],
        [0.8415],
        [1.0000]], grad_fn=<SumBackward1>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 上方的方法已经实现了掩码注意力权重，但是使用了两次归一化，比较复杂。所以还有更加高效的方法 -- 在第一次归一化之前，就加上掩码

In [58]:
mask = torch.triu(torch.ones(context_length,context_length),diagonal = 1) #triu生成上三角矩阵，而tril生成下三角矩阵 。diagonal = 1生成的三角矩阵不包括对角线这一行。
masked = attn_scores.masked_fill(mask.bool(),-torch.inf) #mask.bool()将mask转化为一个布尔矩阵，0代表false，1代表true
#attn_scores中对应于mask矩阵true的地方会被后面一个参数，即这里的负无穷给进行置换
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 使用负无穷进行填充，是为了使得后续在训练的时候，使这里的权重不断趋于0.
- 由于softmax的处理公式中，是将该位置的x值作为$e^{x}$进行处理，如果是0，会变成1，参与了计算。如果是负无穷，则变成了0，不影响结果
- 接下来再利用softmax进行归一化
  

In [60]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5,dim = -1) #对masked矩阵进行归一化
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 2.对额外的注意力权重使用dropout

- 应用dropout来避免模型过拟合
- dropout可用于不同的场景 ：1.计算注意力权重后 2.计算注意力权重和values 所得的向量后
- 本节使用前者（更常见），然后dropout率为50%，，后续实际训练的时候，这个比例会降低到大概0.1~0.2

In [62]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) #dropout率为0.5
example = torch.ones(6,6)
print(dropout(example)) #对example矩阵进行dropout
#被dropout的部分会变成0，而剩余部分会被缩放一个比例：1 / dropout率

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [63]:
torch.manual_seed(123)
print(dropout(attn_weights)) #对上面生成的注意力权重进行dropout

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.8966, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4921, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4350, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


- 在不同的操作系统上，dropout生成的结果可能是不同的

### 3.将上述构造一个因果注意力矩阵压缩包装为一个类

In [66]:
batch = torch.stack((inputs, inputs), dim=0)
#原本inputs 是6个维度为3的输入  。 这里的操作是将两个参数矩阵堆叠（上面是堆叠了两个inputs矩阵），dim=0表示按第0维堆叠，构成一个批次中的两份输入。

print(batch.shape) # 2份输入，每份输入6个token，每个token为3维度

torch.Size([2, 6, 3])


In [73]:
class CausalAttention(nn.Module):  #构造因果注意力
    def __init__(self,d_in,d_out,context_length,
                 dropout,qkv_bias=False): #传入参数：输入维度，输出维度，文本长度，dropoutd的比例，是否偏置
        super().__init__() #初始化父类，确保子类正常进行
        self.d_out = d_out
        self.W_query = nn.Linear(d_in,d_out,bias = qkv_bias)  #初始化训练矩阵们
        self.W_key = nn.Linear(d_in,d_out,bias = qkv_bias)
        self.W_value = nn.Linear(d_in,d_out,bias = qkv_bias)
        self.dropout = nn.Dropout(dropout) #按照参数输入的比例，构造一个dropout层
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))
        #↑注册一个不可训练掩码矩阵mask，triu表示上三角矩阵，上三角矩阵除了对角线都变成1
    
    def forward(self, x):
        b, num_tokens, d_in = x.shape # x是三维，b代表一批输入次数，num_tokens代表每一次输入的词数，d_in代表输入向量维度
        keys = self.W_key(x)  #将x映射到键，查询，值 空间。 由于linear线性层的作用，x只有最后一位会被记录为d_in特征维度，而前面几位维度则会被记录为批次维度
        queries = self.W_query(x) #此时他们几个的维度都是(b,num_tokens,d_out)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2)# 注意力分数为query和key矩阵的乘积。这里第一维是批次数量，不动，需要进行乘法操作的是后面的维度，所以只将keys的后面两个维度进行转置
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # :num_tokens是为了处理当批次内数目小于定义的context_length长度时（当处理到最后的批次时容易出现）
        
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1   #对加上掩码后的注意力分数矩阵进行归一化处理
        )
        attn_weights = self.dropout(attn_weights) # 归一化处理后，通过dropout层处理

        context_vec = attn_weights @ values  #最后上下文向量有注意力权重和values相乘获得
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]  #记录输入向量的数目
ca = CausalAttention(d_in, d_out, context_length, 0.0)  #实例化一个因果注意力处理层

context_vecs = ca(batch) #将batch通过这个处理层进行处理

print(context_vecs) #打印返回的上下文向量
print("context_vecs.shape:", context_vecs.shape) 

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


## 6.将单层思考的注意力模型扩展为多层思考

### 1.将多个单层注意力模型堆叠起来

- 多头脑的注意力模型实现的核心，就是同时运行多个单头脑的注意力模型，并且这些模型的的线性预测是不一样的
- 输入->多层注意力模型处理->产生多个上下文矩阵->将这些上下文矩阵的输出维度进行合并。最后所得的d_out实际为 单层的d_out 乘以 num_heads

In [78]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in,d_out,context_length,dropout,qkv_bias) #产生一个列表，列表中每一个元素都是一个因果注意力处理层
            for _ in range(num_heads)]  # _ 表示不需要使用这个索引值，实际上就是重复num_heads的意思
        )

    def forward(self,x):
        return torch.cat([head(x) for head in self.heads],dim=-1)  #当需要处理一个输入的时候，将其通过每一个注意力处理层。并且将最后一维度进行合并（上下文向量的维度）

torch.manual_seed(123)

context_length = batch.shape[1]  # 每一个输入中tokens的数量
d_in,d_out = 3,2

mha = MultiHeadAttentionWrapper(
    d_in,d_out,context_length,0.0,num_heads=2 #调用多层注意力处理，创建一个实例。现在这个模型有2层思考
)

context_vecs = mha(batch) #将上面的batch输入进行处理(原大小为2,6,3），单层输出为2

print(context_vecs)
print("context_vecs.shape:",context_vecs.shape)  #注意这里输出不再是2,6,2,  而是 2,6,4了

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


### 2.利用单个权重矩阵的分割来实现多头脑思考
- 1中的方法直观且已经包装好了，但是它对于每一头脑，都要重新创建key，query，value矩阵，变量使用较多，效率低。
- 下面写MultiHeadAttention 函数来实现相同的效果。
- 并且这个函数内，不需要再将每单个注意力层进行合并，而是只创建当注意力训练矩阵，并将它分割成单个向量，交给每一个注意力层处理

In [85]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):  #对类进行初始化
        super().__init__()  
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"  #避免错误，输出数量必须要能够被处理头数整除

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # 将参数中的输出维度进行处理，获取其中每个头的特征维度

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias) #创建单个query，key，value训练矩阵
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # 用于合并多个输出的线性映射层
        self.dropout = nn.Dropout(dropout) #dropout层
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1) 
        )  #创建mask掩码

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # 形状: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        #  通过添加了一个num_heads维度，来隐式地分解矩阵
        # 将最后一个维度展开 (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # 将1,2个维度进行转置: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2) #注意索引是从0开始的
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        #  计算缩放过后的注意力分数（按照因果掩码的方式）
        attn_scores = queries @ keys.transpose(2, 3)  # 将每个头内的数据进行点积运算

        # 初始化掩码矩阵，并且转化为bool型。这里需要的矩阵大小是：每一输入中词元的数量
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # 用掩码处理注意力分数
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)  #归一化成注意力权重
        attn_weights = self.dropout(attn_weights)  #通过dropout层进行处理

        # 形状: (b, num_tokens, num_heads, head_dim) 
        context_vec = (attn_weights @ values).transpose(1, 2)  #计算上下文向量，并将矩阵的形状进行恢复。
        
        # C将多个头的数据合并, 其中 self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out) #contiguous() 确保 context_vec 在内存中是连续的，这样 view 才能正确地重构形状
        context_vec = self.out_proj(context_vec) # 通过线性层对上下文向量进行投影，最后的形状是(b,num_tokens,d_out)
        #view只用于改变矩阵的形状，而out_porj最后进行了一个线性变化，整合了每一个头的信息。

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 上面这个类和上上个类实现的功能是一样的，但是它的效率会更高一些
- 输出结果不同，是因为初始化的权重训练矩阵不同，但是他们都是可以有效使用的
- 上面最后用了out_proj线性层进行了处理，但是他没有改变矩阵的维度什么的，它是一个惯例写法。近来有研究表明，这一句可以去掉而不影响整个模型的效果
  
- 如果想要实现一个更加紧凑且高效的多头脑处理，可以考虑使用pytorch自带的  [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 类

- - 对上面的模块进行一些简化分析

In [92]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4) 
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))  #对于 a @ a.transpose的结果（模拟用于计算query矩阵）

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 这里pytorch中的矩阵乘法，会处理四维矩阵，将第最后两维度的内容相乘，然后对每个head进行重复

In [97]:
first_head = a[0,0, :, :] #提取了第一个批次中，第一个头的所有行和列进行计算
first_res = first_head @ first_head.T
print("First head:\n",first_res)

second_head = a[0,1, :, :] #提取第一个批次中，第二个头的所有行和列进行计算 
second_res = second_head @ second_head.T
print("\nSecond head:\n",second_res)

#上述俩位的形状都是(nums_tokens,nums_tokens)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


## 附录： 关于buffer
- 上面在创建mask掩码的时候，用到了register_buffer。这是pytorch提供的。它表示这部分内容是不会被更新的。它不参与训练过程，但却能够在forward中使用
- 并且，如果不使用buffer，当将这个类传输到gpu的时候，类中的这个矩阵还留在cpu，会导致计算出现问题。
- 而使用buffer，会将这个矩阵一同传到gpu中，不需要对这个模块进行单独的处理

- 相较于普通的tensor张量，buffer张量还可以被包含在一个模型的state_dict中
- state_dict可以使用torch.save()方法，将数据保存到文件中。后面可以再使用torch.load()加载，load_state_dict()方法恢复模型数据。
- 如果不使用buffer，你原先对mask掩码进行改变的话，之后再加载的时候，也不会变成你改变后的版本，而只传入了原版本。